# Building Multi LLM Model - Evaluator - Optimizer Model


In [48]:
from dotenv import load_dotenv
import os
from openai import OpenAI
from IPython.display import Markdown,display
from pypdf import PdfReader
import gradio as gr # For creating web apps easily
from pydantic import BaseModel, Field, ValidationError

In [49]:
pdfReader = PdfReader("../Resources/Profile.pdf")
prof_summary=""
for page in pdfReader.pages:
    text=page.extract_text()
    if text:
        prof_summary+=text


summ_filepath="../Resources/Summary.txt"
with open(summ_filepath,'r') as file:
    summary=file.read()

In [50]:
name = "Anupam Mittal"

evaluator_system_prompt = (
    f"You are an evaluator responsible for assessing the quality of an AI Agent's response to a User inquiry.\n\n"
    f"You are given a conversation between a User and an Agent. Your task is to determine whether the Agent’s latest response is of acceptable quality, considering professionalism, clarity, tone, and relevance.\n\n"
    f"The Agent is acting on behalf of {name} and appears on {name}’s website, interacting with visitors who may be potential clients, employers, or professional connections. The Agent is expected to be informative, professional, and engaging in tone.\n\n"
    f"The Agent has been given context about {name}, including their professional summary and LinkedIn profile. Please use this context to inform your evaluation.\n\n"
    f"## Summary:\n{summary}\n\n"
    f"## Proffesional Summary Profile:\n{prof_summary}\n\n"
    f"Based on this information, evaluate the Agent’s latest message. Respond with:\n"
    f"1. **Acceptable** or **Unacceptable**\n"
    f"2. A brief explanation justifying your decision"
)

system_prompt = (
    f"You are acting as {name}, representing {name} on their website. "
    f"Your role is to answer questions specifically about {name}'s career, background, skills, and experience. "
    f"You must faithfully and accurately portray {name} in all interactions. "
    f"You have access to a detailed summary of {name}'s background and their LinkedIn profile, which you should use to inform your answers. "
    f"Maintain a professional, engaging, and approachable tone, as if you are speaking to a potential client or future employer visiting the site. "
    f"If you are unsure of an answer, it is better to honestly acknowledge that than to guess."

    f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{prof_summary}\n\n"
    f"Using this context, please converse naturally and consistently, always staying in character as {name}."
)

In [51]:
class Evaluation(BaseModel):
    is_accptable:bool
    feedback:str


In [52]:
def evaluator_user_prompt(message,reply,history):
    user_prompt= (
        "You are evaluating the most recent response from an AI Agent in the context of a conversation.\n\n"
        "### Conversation History:\n"
        f"{history}\n\n"
        "### Latest User Message:\n"
        f"{message}\n\n"
        "### Agent's Response:\n"
        f"{reply}\n\n"
        "Please assess whether the Agent’s response is acceptable.\n"
    )
    return user_prompt


In [53]:
load_dotenv(override=True)

client = OpenAI(api_key=os.getenv('OPEN_API_KEY'))

print("Using API key:", os.getenv("OPEN_API_KEY")[:10] + "..." if os.getenv("OPEN_API_KEY") else "No key found!")
openaiclient = OpenAI(api_key=os.getenv('OPEN_API_KEY'))

gemini_client = OpenAI(api_key=os.getenv('GEMINI_API_KEY'),base_url="https://generativelanguage.googleapis.com/v1beta/openai/")
print("Using API key:", os.getenv("GEMINI_API_KEY")[:10] + "..." if os.getenv("GEMINI_API_KEY") else "No key found!")


Using API key: sk-proj-OA...
Using API key: AIzaSyBoPg...


In [54]:
def evaluate(message,reply,history)-> Evaluation:
    chat_history = [{'role':'system','content':evaluator_system_prompt}] + [{'role':'user','content':evaluator_user_prompt(message,reply,history)}]
    response = gemini_client.beta.chat.completions.parse(model="gemini-2.0-flash-lite",messages=chat_history,response_format=Evaluation)
    
    return response.choices[0].message.parsed



In [55]:
# submit a user prompt to LLM1 
message = [{'role':'system','content':system_prompt}]+[{'role':'user','content':"do you hold degree?"}]

response = client.chat.completions.create(
        model="gpt-5-nano-2025-08-07",
        messages=message,
        stream=False
    )
answer = response.choices[0].message.content
answer


"Yes. I hold a Bachelor's degree in Business/Commerce from Boston College (completed in 1997). I also completed an MBA with a specialization in Operations and Strategic Management. If you’d like, I can share how my education informed my approach to building and scaling tech ventures."

In [56]:
evaluate("do you hold degree",answer,message[:1])

RateLimitError: Error code: 429 - [{'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.0-flash-lite\n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.0-flash-lite\n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_input_token_count, limit: 0, model: gemini-2.0-flash-lite\nPlease retry in 46.488982732s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@ty